## Задание

Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы) с сайта superjob.ru 
и hh.ru. Приложение должно анализировать несколько страниц сайта(также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:

*Наименование вакансии

*Предлагаемую зарплату (отдельно мин. отдельно макс. и отдельно валюту)

*Ссылку на саму вакансию

*Сайт откуда собрана вакансия

По своему желанию можно добавить еще работодателя и расположение. 
Данная структура должна быть одинаковая для вакансий с обоих сайтов. 
Общий результат можно вывести с помощью dataFrame через pandas.

In [ ]:
import requests
import json
from pprint import pprint
from bs4 import BeautifulSoup as bs
from lxml import html
import pandas as pd
import re

In [2]:
request = 'python'

In [55]:
page = 0
max_page = 0

In [6]:
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36'}


In [56]:
vacancy_link_data = []
vacancy_name_data = []
vacancy_minsalary_data = []
vacancy_maxsalary_data = []
vacancy_currancy_data = []
vacancy_salary_data = []

In [57]:
while max_page != 40:
    link = f'https://hh.ru/search/vacancy?area=1&st=searchVacancy&text={request}&page={page}'
    response = requests.get(link, headers=header).text
    soup = bs(response,'lxml')
    vacancy_block = soup.find_all('div',{'class':'vacancy-serp'})[0]
    vacancy_list = vacancy_block.find_all('div',{'class':'vacancy-serp-item'})
    page += 1
    max_page += 1
    for vacancy in range(len(vacancy_list)):
        name1 = vacancy_block.find_all('span',{'class':'g-user-content'})[vacancy]
        name2 = name1.findChild()
        vacancy_name = name2.getText()
        vacancy_name_data.append(vacancy_name)
        vacancy_data={}
        vacancy_link1 = vacancy_block.find_all('span',{'class':'g-user-content'})[vacancy]
        vacancy_link = vacancy_link1.find('a')['href']
        vacancy_link_data.append(vacancy_link)
        salary1 = vacancy_block.find_all('div',{'class':'vacancy-serp-item__sidebar'})[vacancy]
        salary2 = str(salary1.findChild(recursive=False))
        salary3 = re.sub(r'\<[^>]*\>', '', salary2)

        if salary3 == None:
            salary = 0
        else:
            salary = salary3
        vacancy_salary_data.append(salary)    

In [58]:
len(vacancy_salary_data)
        

2010

In [59]:
len(vacancy_list)

50

In [60]:
data = {'Vacancy Name': vacancy_name_data, 'Link': vacancy_link_data, 'Salary': vacancy_salary_data, 'Source': 'hh'}
df = pd.DataFrame(data)

In [61]:
df.head()

,Vacancy Name,Link,Salary,Source
0,Инженер-программист,https://hh.ru/vacancy/36349898?query=python,120 000-140 000 руб.,hh
1,Программист Python,https://hh.ru/vacancy/36615032?query=python,None,hh
2,Golang разработчик (Backend),https://hh.ru/vacancy/36631687?query=python,170 000-250 000 руб.,hh
3,Реверс инженер мобильных приложений,https://hh.ru/vacancy/36615038?query=python,,hh
4,Старший программист,https://hh.ru/vacancy/36348947?query=python,230 000-250 000 руб.,hh
